# Segmenting and Clustering Neighborhoods in Toronto Assignment

## Part 1

In [3]:
import pandas as pd
import requests

In [36]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]

df = df[df.Borough != 'Not assigned']
df.reset_index(drop=True, inplace=True)
df.Neighborhood.fillna(df.Borough, inplace=True)
df = df.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()

df.shape

(103, 3)

## Part 2